In [ ]:
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Flatten, LSTM, Dropout
from keras.utils import np_utils
from sklearn.preprocessing import MinMaxScaler
import math
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
test_data = pd.read_csv('test.csv')
test_sequences = test_data['Sequence']
chars = [',', '-', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
y_pred = [0] * len(test_sequences)
exceptions = 0
n_train = 100

total_start = time.time()
for (j, sequence) in enumerate(test_sequences[:20]):
    start = time.time()
    if j % 1 == 0:
        print('Processing sequence {0}... '.format(j))
    if len(sequence) < 101:
        continue
    numbers = sequence.split(',')
    if len(numbers) < 4:
        continue
    len_n_train = 10
    if len(numbers) < 11:
        len_n_train = len(numbers) // 2
    len_sequence = [len(item) for item in numbers]
    len_x_train = []
    len_y_train = []
    for i in range(len(numbers) - len_n_train - 1):
        len_x_train.append(numbers[i:i + len_n_train])
        len_y_train.append(numbers[i + len_n_train])
    print('formed len train data')
    len_x_train = np.reshape(len_x_train, (-1, len_n_train, 1))
    len_y_train = np_utils.to_categorical(len_y_train)
    len_model = Sequential()
    len_model.add(LSTM(4, input_shape=(len_x_train.shape[1], len_x_train.shape[2])))
    len_model.add(Dropout(0.2))
    len_model.add(Dense(len_y_train.shape[1], activation='softmax'))
    len_model.compile(loss='categorical_crossentropy', optimizer='adam')
    len_model.fit(len_x_train, len_y_train, epochs=20, verbose=1)    
    print('len model fitted')
    len_prob = len_model.predict(np.reshape(numbers[-len_n_train:], (1, len_n_train, 1)), verbose=1)
    predicted_len = np.argmax(len_prob)
    print('predicted_len:', predicted_len)

    sequence += ','
    x_train = []
    y_train = []
    for i in range(len(sequence) - n_train - 1):
        x_train.append([char_to_int[char] for char in sequence[i:i + n_train]])
        y_train.append(char_to_int[sequence[i + n_train]])

    print('formed train data')
    x_train = np.reshape(x_train, (-1, n_train, 1))
    y_train = np_utils.to_categorical(y_train)
    model = Sequential()
    model.add(LSTM(256, input_shape=(x_train.shape[1], x_train.shape[2])))
    model.add(Dropout(0.2))
    model.add(Dense(y_train.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    model.fit(x_train, y_train, epochs=20, verbose=1)
    print('model fitted')

    word = ''
    seq_to_predict = [char_to_int[char] for char in sequence[-n_train:]]
    for i in range(predicted_len):
        try:
            x_test = np.reshape(seq_to_predict, (1, n_train, 1))
            prediction = model.predict(x_test, verbose=1)
            pred_int = np.argmax(prediction)
            pred_char = int_to_char[pred_int]
            if pred_char == ',':
                break
            word += pred_char
            seq_to_predict.append(pred_int)
            seq_to_predict = seq_to_predict[-n_train:]
        except Exception as e:
            print(e)
    y_pred[j] = word
    end = time.time()
    if j % 1 == 0:
        print('Predicted: {0}. Time: {1}'.format(word, end - start))

total_end = time.time()
print('Total time: {0}'.format(total_end - total_start))